In [1]:
import os
import zipfile
import subprocess
import tempfile
import shutil
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

# Load API key and instantiate OpenAI client
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API key not found. Ensure it is set in the .env file.")

client = OpenAI(api_key=api_key)

##############################
# Existing Functions (File-based)
##############################

def process_file(file):
    """Extract text from the uploaded file (ZIP or plain text)."""
    if not file:
        return "No file uploaded."
    filename = file.name
    ext = os.path.splitext(filename)[1].lower()

    if ext == ".zip":
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            file_list = zip_ref.namelist()
            contents = []
            for f in file_list:
                if "readme" in f.lower() or os.path.splitext(f)[1].lower() in ['.txt', '.csv', '.log']:
                    with zip_ref.open(f) as f_in:
                        try:
                            text = f_in.read().decode("utf-8")
                        except UnicodeDecodeError:
                            text = f_in.read().decode("latin-1")
                        contents.append(text)
            return "\n".join(contents) if contents else "No readable text found in the ZIP."
    elif ext in ['.txt', '.csv', '.log']:
        with open(filename, "r", encoding="utf-8") as f:
            return f.read()
    else:
        return "File type not supported for text extraction."

def get_command_lines_from_readme(file):
    """Use GPT to extract only the command-line instructions from the README-like text."""
    extracted_text = process_file(file)
    if not extracted_text or "No file" in extracted_text or "not supported" in extracted_text:
        return extracted_text

    prompt = (
        "You are given the contents of a README file below. "
        "Please extract and print only the command-line instructions. "
        "Ignore all other text. Remove triple backticks, etc.\n\n"
        f"{extracted_text}"
    )

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": prompt}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    extracted_commands = response.choices[0].message.content
    # Remove any leftover markdown formatting (e.g., triple backticks)
    cleaned_commands = extracted_commands.replace("```bash", "").replace("```", "").strip()
    return cleaned_commands

def run_commands_live(command_text):
    """
    A generator that executes shell commands dynamically and streams output live,
    while preserving the full log history.
    """
    commands = command_text.strip().splitlines()
    error_detected = False
    log_history = ""  # Accumulate logs

    for cmd in commands:
        if not cmd.strip():
            continue  # Skip empty lines

        log_history += f"\n---\nRunning command: {cmd}\n"
        yield log_history  # Yield the full log so far

        try:
            process = subprocess.Popen(
                cmd,
                shell=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )

            # Stream stdout line-by-line
            for line in iter(process.stdout.readline, ''):
                log_history += line
                yield log_history

            # Check stderr for errors
            stderr_output = process.stderr.read()
            if stderr_output:
                log_history += stderr_output
                error_detected = True
                log_history += "\n❌ Error detected. Stopping execution.\n"
                yield log_history
                break

            exit_code = process.wait()
            if exit_code != 0:
                error_detected = True
                log_history += f"\n❌ Error: Command failed with exit code {exit_code}. Stopping.\n"
                yield log_history
                break

        except Exception as e:
            error_detected = True
            log_history += f"\n❌ Exception: {str(e)}\n"
            yield log_history
            break

    if not error_detected:
        log_history += "\n✅ Successfully executed all command lines without any errors.\n"
        yield log_history

def run_commands_with_extraction(file):
    """
    1. Extract commands from the README via GPT
    2. Yield live output as commands run
    """
    commands = get_command_lines_from_readme(file)
    if (not commands or "No file uploaded" in commands or 
        "not supported" in commands or "No readable text" in commands):
        yield commands
        return

    yield from run_commands_live(commands)

##############################
# New Functions for GitHub URL-based Execution
##############################

def clone_repo(github_url):
    """
    Clones the GitHub repository into a temporary directory.
    Returns the path to the cloned repository.
    """
    temp_dir = tempfile.mkdtemp()
    cmd = f"git clone {github_url} {temp_dir}"
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        shutil.rmtree(temp_dir)
        raise Exception(f"Failed to clone repository: {result.stderr}")
    return temp_dir

def get_command_lines_from_text(text):
    """
    Uses GPT to extract command-line instructions from a given text.
    """
    if not text:
        return "No text provided."
    prompt = (
        "You are given the contents of a README file below. "
        "Please extract and print only the command-line instructions. "
        "Ignore all other text. Remove triple backticks, etc.\n\n"
        f"{text}"
    )
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    extracted_commands = response.choices[0].message.content
    cleaned_commands = extracted_commands.replace("```bash", "").replace("```", "").strip()
    return cleaned_commands

def run_from_github(github_url):
    """
    Clones a GitHub repository, reads the README file, extracts command-line instructions,
    executes them live, and returns the full log.
    """
    log_history = ""  # Initialize log accumulator

    # Step 1: Clone the repository
    try:
        log_history += f"Cloning repository: {github_url}\n"
        yield log_history  # Update UI
        repo_dir = clone_repo(github_url)
        log_history += f"Repository cloned to {repo_dir}\n"
        yield log_history
    except Exception as e:
        log_history += f"❌ Error cloning repository: {str(e)}\n"
        yield log_history
        return

    # Step 2: Locate the README file
    readme_path = None
    for candidate in ["README.md", "readme.md", "Readme.md"]:
        path = os.path.join(repo_dir, candidate)
        if os.path.exists(path):
            readme_path = path
            break
    if not readme_path:
        log_history += "❌ No README file found in the repository.\n"
        yield log_history
        shutil.rmtree(repo_dir)
        return

    log_history += f"Found README: {readme_path}\n"
    yield log_history

    # Step 3: Read the README content
    try:
        with open(readme_path, "r", encoding="utf-8") as f:
            readme_content = f.read()
    except Exception as e:
        log_history += f"❌ Error reading README: {str(e)}\n"
        yield log_history
        shutil.rmtree(repo_dir)
        return

    # Step 4: Extract command-line instructions using GPT
    log_history += "Extracting command lines from README...\n"
    yield log_history
    commands = get_command_lines_from_text(readme_content)
    log_history += f"Extracted commands:\n{commands}\n"
    yield log_history

    # Step 5: Execute the commands live and stream output
    log_history += "Executing extracted commands...\n"
    yield log_history
    for output in run_commands_live(commands):
        log_history += output  # Append output dynamically
        yield log_history  # Yield the full updated log

    # Cleanup the cloned repository
    shutil.rmtree(repo_dir)
    log_history += "Cleaned up cloned repository.\n"
    yield log_history  # Final log update


##############################
# Gradio Interfaces
##############################

# Existing file-based interface (for uploading README or ZIP)
file_demo = gr.Interface(
    fn=run_commands_with_extraction,
    inputs=gr.File(label="Upload README or ZIP"),
    outputs=gr.Textbox(label="Live Execution Log", lines=15),
    description="Upload a README or ZIP. We'll extract command lines using GPT, then run them line-by-line."
)

# New GitHub URL-based interface
github_demo = gr.Interface(
    fn=run_from_github,
    inputs=gr.Textbox(label="GitHub Repository URL", placeholder="https://github.com/username/repo.git"),
    outputs=gr.Textbox(label="Live Execution Log", lines=15),
    description="Enter a GitHub URL. The bot will clone the repo, extract command lines from the README, execute them, and return the status."
)

# Combine both interfaces in tabs for convenience
with gr.Blocks() as demo:
    gr.Markdown("# Advanced Command Executor Bot")
    gr.Markdown("Choose an input method:")
    with gr.Tabs():
        with gr.TabItem("File Upload"):
            file_demo.render()
        with gr.TabItem("GitHub URL"):
            github_demo.render()

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://9759cf02b63b689d5d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
